**Transformer实战（下）**

你是否已经阅读过《Transformer实战（上）》的部分？如果你阅读过，那你其实已经掌握了使用transformer进行NLP预测的大部分精髓，无需再阅读该文档。相比起复杂的机器翻译与股价预测大案例，这一篇Transformer实战（下）中的情感分类与词性标注任务则是更为基础、更为经典的NLP任务（encoder-only的分类任务）。词性标注和情感分类任务都属于NLP发展早期的任务，如果你没有阅读过《实战（上）》的内容，那这一篇中相对轻便的两个案例也是很好的Transformer实战入门选择。

在这一篇中，我将不会再进行像《实战（上）》篇那样详细的NLP技术细节梳理，而是会着重于帮助你复现可以跑通这两个场景的代码，让你可以将代码即拿即用。在这一篇中你所需的任何知识，你都可以在我们已有的NLP课程中找到，如果有任何困惑，欢迎在群内进行交流。

# Transformer案例实战 之 情感分类

极性情感分类是一种利用计算机程序通过分析文本内容来判定其情感倾向（如正面、负面或中性）的技术，它是自然语言处理领域中的一项基本任务，也是因其广泛的应用场景而备受关注的领域。这个任务涉及到从文本数据中提取情感信号，并将这些信号准确地映射到预定义的情感类别中，同时尽可能地捕捉文本中的情感细节和语境变化，而这些文本数据可能来自网站评论区、社交媒体平台或专门为情感分析任务收集的语料库。数据集中的每个样本都已经被人工标注好情感极性，这为训练监督学习模型提供了基础。总而言之，情感分类是自然语言处理领域中的一个重要而典型的研究方向，广泛应用于客户服务、社交媒体分析和市场研究等领域。

在使用深度学习技术进行情感分类的各种实践中，Transformer被认为是非常有效的模型。通过训练大量的文本数据，Transformer能够学习复杂的情感表达模式，它能够理解和分类各种复杂的情绪表达，适应不同的语言风格，并处理含糊或隐晦的情感表达。对于Transformer模型来说，情感分类是一个典型的分类任务。这意味着模型需要接收一系列的输入（如文本数据），并输出一个分类结果（情感类别）。Transformer中的自注意力机制允许模型在分类时能够关注到输入文本中的所有词语，帮助模型理解更复杂的情感表达和语境依赖，使得分类结果更加准确和具有深度；同时，在情感分类任务中，Transformer中的编码器用于理解输入的文本，而输出层则直接基于编码后的表示来判定情感类别，因此情感分类是一个只使用编码器、而不是用解码器的架构。在Tansformer用于情感分类任务时，每一层都进一步提炼和传递信息，增强了模型在处理复杂文本时的能力。

## 情感分类任务的基本流程

在今天的项目中，我们将完成一个英文文本的情感分类任务。通常来说，一个情感分类任务需要覆盖至少如下的流程：

### 数据准备与经典情感分类数据集

- 一、数据准备

对于任何NLP任务来说，我们都需要对数据进行详细的处理、毕竟算法本身无法处理文字数据，而语言数据同时带有时序属性和文字属性，具有复杂的处理流程，其中中文又比英文更加复杂。

1. **数据加载**：本次我们准备的是基于英文文本的情感分类任务，我们使用了自然语言库NLTK中的sentence_polarity数据集。NLTK（Natural Language Toolkit）是一个强大的Python库，专门用于处理人类语言数据，广泛应用于自然语言处理（NLP）研究和教育，而本次我们所使用的sentence_polarity数据则是一个经典的英文情感分类数据集。在深度学习的世界中，还有大量以中文为主的情感分类数据集 ↓

> **中文情感分析综合数据集** (Chinese Sentiment Analysis Dataset, ChnSentiCorp)：这是一个广泛使用的中文情感分类数据集，包含酒店、书籍和电子产品的评论，这些评论被标注为正面或负面。

> **豆瓣电影评论数据集**：你可以在github找到这个数据集，这个数据集包含大量来自豆瓣网站的电影评论，这些评论被用户标记为推荐（正面）或不推荐（负面）。

> **微博情感分析数据集**：你依然可以从github找到这个数据集，这是从新浪微博收集的数据，包括带有情感倾向的微博文本，常用于研究社交媒体上的情感表达。

当然，也还有很多其他语言的情感分类数据集——

> **IMDb电影评论数据集 (IMDb Movie Review Dataset)**：包含来自IMDb网站的50,000条电影评论，分为正面和负面两类，是进行情感分析的一个标准数据集。

> **亚马逊商品评论数据集 (Amazon Product Review Dataset)**：包含数百万用户对亚马逊商品的评论，这些评论被标记有星级，通常用星级来推断评论的情感倾向。

> **斯坦福情感树库 (Stanford Sentiment Treebank, SST)**：包含电影评论的数据集，特点是每个句子都被分析成语法树，每个节点（句子、短语、单词）都被标注情感，非常适合深入研究语义上的情感分析。

> **Twitter情感分析数据集**：包含从Twitter抓取的推文，这些推文被标注为正面、负面或中性，用于研究社交媒体文本的情感倾向。

不难发现，情感分类的数据集基本都来自于对于公开评论、网络评论的收集。随着NLP技术的发展，现在已经较少有仅仅针对情感分析进行学术研究的项目了，情感分类被认为是“自然语言理解”的一个子板块。在NLP的领域，我们会要求模型对语言的理解能力更强。

### 分词与Huggingface分词器

2. **分词**：在获得数据集之后，首先我们要进行分词。对于英文、法文这些天然就有空格来进行分词的语言来说，是否进行分词或许没有那么重要，但对于中文、日语这些没有天然分分隔的语言来说，分词是无法跳过的步骤。分词，又叫做Tokenizer，意思是将文字分成最小语义单元Token。在执行生成式任务时，我们会很关注将句子分割成一个个Token，但是在情感分类这样的任务中，我们进行分词只是为了更方便地进行embedding、并将句子更顺畅地输入到模型中。在本次的案例中，因为我们使用的是英文数据集，因此我们不必进行刻意的分词。
> 在之前的课程中，我们曾经介绍过空格分词、jieba分词等基本的分词方法，但随着深度学习和预训练语言模型的发展，现在的分词工具更多地依赖于预训练的模型，如 Hugging Face 的 transformers 库中的分词器（Tokenizer）。这些分词器不仅支持分词操作，还可以处理子词（subword）和字符级别的分词。以下是一些著名的分词器：<br><br>
> **1. BERT Tokenizer**<br><br>
> BERT（Bidirectional Encoder Representations from Transformers）使用 WordPiece 分词算法，它将文本分割成子词单元，以解决稀有词汇问题。<br><br>
> **2. GPT Tokenizer**<br><br>
> GPT（Generative Pretrained Transformer）使用的是 Byte Pair Encoding (BPE) 分词算法，主要用于处理英语等语言。<br><br>
> **3. RoBERTa Tokenizer**<br><br>
> RoBERTa（Robustly optimized BERT approach）与 BERT 类似，也使用了 BPE 分词算法，但在训练过程中进行了优化。<br><br>
> **4. XLNet Tokenizer**<br><br>
> XLNet 是一种结合了自回归和自编码方法的语言模型，其分词器也使用了 SentencePiece。<br><br>
> **5. T5 Tokenizer**<br><br>
> T5（Text-To-Text Transfer Transformer）使用了 SentencePiece 分词算法，并将所有 NLP 任务都视为文本到文本的转换。

In [ ]:
# 注意，以下代码需要魔法才能运行
# 注意，以下代码需要魔法才能运行
# 注意，以下代码需要魔法才能运行

# 虽然是Huggingface库，但实际在导入的时候库的名字却是transformer
# 在transformer架构讲解中我们有详细讲解Huggingface的调用，大家可以回看transformer章节
# Huggingface的调用不仅需要魔法、而且还需要将巨量模型从远程下载到本地
# 这个过程没有足够好的镜像网站支持，因此只能依靠强力并且有巨大流量空间的魔法
# 在之前的课程中我曾为大家准备了一系列下载好的Huggingface模型
# 都在咱们课程资料中，大家可以去找找

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer.tokenize("Hello, how are you?")
print(tokens)

from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokens = tokenizer.tokenize("Hello, how are you?")
print(tokens)

from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokens = tokenizer.tokenize("Hello, how are you?")
print(tokens)

from transformers import XLNetTokenizer

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
tokens = tokenizer.tokenize("Hello, how are you?")
print(tokens)

from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small')
tokens = tokenizer.tokenize("Hello, how are you?")
print(tokens)

### 词典构建过程中的3大核心问题

3. **构建词汇表与词典**：当我们将句子进行分词之后，下一步需要构建当前数据集所构成的词汇表与词典。所谓词汇表，就是包含了这个数据集中所有词的列表，而所谓的词典，是将数据集中的每一个词与一个独有的索引相连的结构，基本等同于对文字进行频率/数字的编码后、将文字与编码匹配在一起存储的状态。
> **在使用pytorch实现Transformer的时候，nn.Transformer中是不包括位置编码、掩码等encoder、decoder之前的结构的；因此我们首先要将数据传入embedding和位置编码结构，然后才会将数据传入Transformer，我们在构建词典后、真正获得的数据是与词典紧密相连的单词索引，而非直接放入Transoformer的数据**。我们通常会基于相同的词典将训练集、测试集进行索引，同时编码器和解码器都会基于相同的词典将单词索引映射为embedding后的向量，以确保输入和输出的一致性。在本次案例中，我们数据处理的重点也将会放在词典与词汇表的构建上。<br><br>
> 关于词典构造，本次我们将详细讨论和讲解几个关键问题。<br>
> - **经过词典所构建的数据（也就是单词索引）一般是什么样的？**<br><br>
> 源语言的单词索引序列通常是二维的数据结构。每个维度代表的含义如下：第一维代表批次中的句子数量（也就是batch size）。这允许模型同时处理多个句子，提高处理效率。第二维代表每个句子中的单词索引。句子长度可以是固定的（通过截断或填充来标准化），这样每个批次的数据形状才能保持一致。每个单词索引是一个整数，表示在词典中的位置。这里有一个简单的Markdown代码例子来表示源语言的单词索引序列。假设我们有以下中文句子，已经经过分词和转换为索引：<br><br>
> 句子1: "我 爱 北京 天安门"<br><br>
> 句子2: "北京 是 中国 的 首都"<br><br>
> 假设这两个句子是整个数据集，那我们首先要创造一个词汇表，该词汇表中包含了数据集中所有出现过的单词：<br><br>
> idx_to_token = ["我","爱","北京","天安门","是","中国","的","首都"]<br><br>
> 我们常常用变量idx_to_token来表示词汇表，因为只要我们索引词汇表，就可以得出具体的token：

In [25]:
idx_to_token = ["我","爱","北京","天安门","是","中国","的","首都"]

In [27]:
idx_to_token[3] #可以通过索引取出具体的Token，所以叫做idx_to_token

'天安门'

假设词典中每个词对应的索引如下：

我 -> 1, 爱 -> 2, 北京 -> 3, 天安门 -> 4, 是 -> 5, 中国 -> 6, 的 -> 7, 首都 -> 8

如果将这些句子转换为单词索引序列，并考虑到填充（以保持统一长度，这里填充的索引为0），则可得到：

In [ ]:
句子1索引: [1, 2, 3, 4]

句子2索引: [3, 5, 6, 7, 8]

为了在一个批次中处理，假设填充每个句子到最大长度5，则单词索引序列为：

In [ ]:
[[1, 2, 3, 4, 0],  # 句子1填充后

 [3, 5, 6, 7, 8]]  # 句子2

结构为（2,5），代表一共有2个句子，句子的最大长度为5。

> - **词典构造的过程中有哪些关键注意事项可能需要考虑？**<br><br>
> 第一个需要考虑的问题是**陌生词**，也叫未知词。在进行深度学习算法预测的时候，只有在词汇表中的词才能够被进行编码，但是在预测中、可能会出现很多不常用的词、拼写错误的词、甚至可能因为训练数据集比较小、所以在测试数据中出现陌生的词。当出现这种情况时，编码就可能会发生错误，而且如果每次都给不常用词、或者拼写错误的词分配唯一的索引、可能会导致词汇表非常大且稀疏。因此在我们构建词汇表的时候，我们要使用字符串"\<unk>"（unknown）来代表模型在训练过程中从未遇到过的词，以减少词汇表的大小，提高模型的泛化能力。有了"\<unk>"的存在，模型就能够处理在训练中未遇到过的词。<br><br>
> 第二个需要考虑的问题是**词汇表/词典的质量**。原则上来说，词汇表中应该包括语料中出现过的所有词，但这其实是一个低效并且甚至可能损害算法的语言理解能力的状态，因为一个NLP项目涉及到的语料越多时、语料中的垃圾信息就可能越多。例如，拼写错误的可能性会增加、不常见的词的词可能会变得更多、特殊符号甚至广告等信息混入的可能性就变得更大，事实上在大部分的语料中，只有少量高频且有价值的词汇是值得NLP算法进行学习的。因此在针对大型NLP数据构建词汇表和词典时，我们要重视词汇表的质量，需要对词汇表进行压缩和筛选。因此我们有着以下的筛选方法——<br><br>
>> 1. 停用词过滤<br>
>> 停用词（stop words）是一些在文本中频繁出现但对语义贡献较少的词，如“the”、“is”、“in”等。过滤掉这些停用词可以减少词汇表的大小，同时不影响文本的语义理解。<br><br>
>> 2. 词性过滤<br>
>> 根据具体任务的需求，只保留特定词性的词汇。例如，对于情感分析任务，形容词和副词可能更重要，而名词和动词则在信息检索任务中更为关键。<br><br>
>> 3. 词长度过滤<br>
>> 过滤掉过短或过长的词汇。例如，长度为1或2的词汇可能是噪音，而过长的词汇可能是拼写错误。<br><br>
>> 4. 字符类型过滤<br>
>> 根据字符类型进行过滤。例如，只保留字母或数字，过滤掉包含特殊字符的词汇。这可以有效去除文本中的特殊符号和广告信息。<br><br>
>> 5. 词汇标准化<br>
>> 使用词干提取（stemming）或词形还原（lemmatization）将不同形式的同一词汇标准化为一个基本形式。例如，将“running”和“ran”都转换为“run”，从而减少词汇表的大小。<br><br>
>> 6. 频率截断<br>
>> 除了简单的词频筛选，还可以设置上限和下限频率阈值。即保留在一定频率范围内的词汇，过高频率的词汇也可以视为停用词进行过滤。<br><br>
>> 7. 领域相关性筛选<br>
对于特定领域的NLP任务，可以根据领域词典或知识库进行筛选，只保留与领域相关的词汇。例如，医学文本处理时，只保留医学术语。<br><br>
>> 8. 基于统计和信息论的方法<br>
>> 使用更复杂的统计方法，如互信息（Mutual Information）、卡方检验（Chi-Square Test）、点互信息（Pointwise Mutual Information, PMI）等，评估词汇的重要性和关联性，进行筛选。<br><br>
>> 9. 基于预训练模型的词汇筛选<br>
>> 使用预训练语言模型的词嵌入，计算词汇的语义相似性，将相似性较高的词汇进行合并或筛选，保留重要的词汇。<br><br>
>>这些方法可以单独使用，也可以组合使用，以构建一个高质量、精简且适用于具体任务的词汇表。选择合适的筛选方法取决于具体的应用场景和任务需求。<br><br>
>>**在本次案例中，我们所使用的方法是词频筛选**。词汇频率筛选在自然语言处理任务中具有重要意义，因为它能够有效地减少噪音，提高模型的训练效率和性能。通过统计每个词汇在文本中的出现频率，我们可以筛选出那些高频且有价值的词汇，避免将稀有词汇引入词汇表。低频词汇往往是拼写错误、特殊符号或不常见的词，这些词汇不仅增加了模型的复杂性，还可能对模型的学习过程产生干扰。通过过滤掉这些低频词汇，我们不仅能够简化词汇表，减少计算资源的消耗，还能使模型更专注于学习有用的词汇和模式，从而提升整体的表现。此外，较小的词汇表意味着更少的参数需要处理，训练速度也因此得到显著提升。因此，词汇频率筛选在构建高质量词汇表时至关重要，是提高模型效率和性能的关键步骤。

4. **数据预处理**：这里的数据预处理并不是常规的填补缺失值等预处理，而是包括了以下的步骤的NLP数据预处理：
> **文本转索引**（根据已存在的词典、将原始数据进行编码）：原始文本通过查找构建好的词典被转换为单词索引序列。这一步是将自然语言中的单词转换为模型可以理解的数值形式。<br><br>
> **索引转嵌入**（将词典编码结果转变为embedding）：这些单词索引接着被用来从嵌入层中获取对应的词向量。这一步是通过词嵌入将每个单词的索引映射到一个高维空间中，以便捕捉和表达词汇的语义和语法属性。<br><br>
> **加入位置编码**：在获取词向量后，模型还会加入位置编码，以引入序列中各单词的位置信息，这对于帮助模型理解词语之间的关系非常重要。<br><br>
> **数据规范化**：对长句子进行裁剪、短句子进行填充，保证所有句子在embedding之前所包含的单词量一致<br><br>
> **设置掩码**：对编码后的数据进行掩码。假设架构有解码器，则需要掩码未来的部分（矩阵的上三角）。假设架构只有解码器，则需要对短句子填充的部分进行掩码，以免给句子带来干扰。<br><br>
对句子进行了这些预处理之后，数据才可以被输入到自注意力机制结构当中。

- 二、模型构建

在之前的课程中，我们详细复现过Transformer架构，因此你对于Transformer结构应该非常熟悉。无论是多么简单的NLP任务，transformer中的零散结构都是需要自己定义的。在情感分类任务中，我们只使用了encoder编码器，因此我们需要定义的结构有：位置编码层、embedding层、包含注意力机制的编码层、打包了多个编码层的编码器、以及对编码器中的输入数据进行掩码的掩码结构。

1. **位置编码**：
在Transformer模型中，位置编码用于向模型提供关于单词在句子中位置的信息，因为模型的自注意力层本身不包含处理序列顺序的机制。位置编码通过将位置信息编码为向量并与词向量相加，使得模型能够考虑到词语的顺序关系，从而理解语言的语法和句子结构。这样的设计允许Transformer有效地处理自然语言，并保持在不同位置的词语能够被区分和正确解释。我们定义了`PositionalEncoding`类为Transformer模型提供位置信息，这对于处理序列数据至关重要。

位置编码的实现原理是完全按照Transformer原始论文对位置编码的需求来实现的，如果你不了解位置编码的基本原理，你是无法理解位置编码的代码本身的。欢迎回看《Transformer原理与实战》部分。

2. **模型定义**：在我们的案例中，我们使用了一个基本的Transformer模型，命名为`Transformer`类，它包括源语言嵌入、目标语言嵌入、位置编码、embedding层、Encoder层、和最终的预测层（线性层）。

- 三、训练过程

Transformer的训练过程对我们来说其实是相对简单和熟悉的。我们需要定义Dataloader等结构，并且需要保持每个batch内的数据长度一致（需要对短的数据进行填充，对长的数据进行裁剪）。除此之外，我们需要定义优化器、进行epoch上的循环等过程。这个流程与其他深度学习算法并无区别，因此对大家来说相对容易。

1. **数据加载器**：在PyTorch定义`DataLoader`用于批量加载数据，以及`collate_fn`函数对数据进行适当处理，确保批次内的数据长度一致。
2. **优化器**：我们选择Adam优化器进行参数优化。
3. **训练循环**：进行模型训练，包括前向传播、损失计算、梯度计算和参数更新。同时设置定期保存模型的逻辑。
4. **可视化与监控**：我们使用了tqdm库来实现训练过程的可视化和监控。

接下来就让我们一步步来完成这个案例。

## 数据准备与数据认识

In [2]:
import math
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
# 用于构建掩码的库
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
# 用于构建词典的库
from collections import defaultdict

- 导入nltk中的数据sentence_polarity来使用

首先要确保你的系统重装有nltk库，如果没有的话你可以使用如下的代码来简单进行安装：

In [ ]:
#!pip install nltk

In [3]:
import nltk

#我们可以将nltk中所带的数据下载到本地来进行调用
#这个过程可能需要梯子
nltk.download('sentence_polarity')

[nltk_data] Error loading sentence_polarity: <urlopen error [WinError
[nltk_data]     10054] 远程主机强迫关闭了一个现有的连接。>


False

NLTK（自然语言工具包）提供了广泛的API，用于处理和分析文本数据。这些API可以分为几个主要板块，每个板块都提供了特定的功能和工具。以下是一些主要的板块和它们的基本功能描述：

1. **应用和工具（Applications and Utilities）**
   - 包含多个小工具和应用程序，如语法分析器、词块分析器和词频应用，帮助用户通过图形界面直观地了解语言处理的不同方面。例如——
   - **`nltk.app.concordance_app`**: 可视化文本中词语的共现情况。
   - **`nltk.app.wordfreq_app`**: 展示词频分析结果。
<br><br>
2. **语法和解析（Parsing and Grammar）**
   - 提供了解析自然语言结构的工具，包括不同类型的解析器（如概率、特征图、递归下降等），以及用于构建和测试语法的模块。例如——
   - **`nltk.parse.stanford`**: 使用Stanford Parser进行句法分析。
   - **`nltk.parse.earleychart`**: 使用Earley算法实现的图表解析器。
<br><br>
3. **语言模型（Language Modeling）**
   - 用于构建和应用统计语言模型，包括n-gram模型和平滑技术，以预测文本中的词序列和概率。例如——
   - **`nltk.lm.preprocessing`**: 为语言模型准备数据。
   - **`nltk.lm.models`**: 包括不同类型的语言模型，如N-gram。
<br><br>
4. **机器学习分类和聚类（Machine Learning for Classification and Clustering）**
   - 包括用于文本分类的各种算法（如朴素贝叶斯、最大熵、决策树等），以及聚类算法，如K-means和EM算法，用于无监督学习和数据分组。例如——
   - **`nltk.classify.naivebayes`**: 实现朴素贝叶斯分类器。
   - **`nltk.cluster.kmeans`**: 实现K-means聚类算法。
<br><br>
5. **词法分析（Tokenization）**
   - 提供文本分割工具，将文本分割成句子、单词或其他有意义的单位，支持多种语言和文本格式。例如——
   - **`nltk.tokenize.punkt`**: 使用Punkt句子分割器进行文本分割。
   - **`nltk.tokenize.treebank`**: 使用Treebank风格的词法分析。
<br><br>
6. **词干提取和词形变化（Stemming and Lemmatization）**
   - 提供用于词形还原和词干提取的工具，减少单词到其基本形式，以便于文本预处理和分析。例如——
   - **`nltk.stem.porter`**: 实现Porter词干算法。
   - **`nltk.stem.wordnet`**: 使用WordNet词形还原器。
<br><br>
7. **语料库和语料库读取器（Corpora and Corpus Readers）**
   - 提供对广泛语料库的访问，以及专门的读取器，用于从不同格式的语料库中提取数据。例如——
   - **`nltk.corpus.reader`**: 提供多种语料库读取器，如平面文本、XML结构等。
   - **`nltk.corpus.util`**: 提供语料库相关的工具函数。
<br><br>
8. **评估和测试（Evaluation and Testing）**
   - 包括用于评估语言模型和解析器性能的工具，以及用于执行语言处理任务的各种测试和验证工具。例如——
   - **`nltk.parse.evaluate`**: 评估解析器的准确率和其他性能指标。
   - **`nltk.metrics.scores`**: 提供多种评估分类器和其他NLP任务的性能的函数。
<br><br>
9. **标注和分类（Tagging and Classification）**
   - 包括为文本分配词性标签和其他标注的工具，如隐马尔科夫模型、感知器和条件随机场等。例如——
   - **`nltk.tag.hmm`**: 使用隐马尔可夫模型进行词性标注。
   - **`nltk.tag.perceptron`**: 使用感知机算法进行标注。
<br><br>
10. **情感分析（Sentiment Analysis）**
    - 提供用于确定文本情感倾向（如积极或消极）的工具，常用于分析社交媒体和消费者评论。例如——
    - **`nltk.sentiment.vader`**: 使用VADER进行情感分析。
    - **`nltk.sentiment.sentiment_analyzer`**: 提供一个情感分析框架。
<br><br>
11. **翻译和跨语言工具（Translation and Cross-Lingual Tools）**
    - 包含用于语言翻译和评估翻译质量的工具，如BLEU和METEOR评分系统。例如——
    - **`nltk.translate.bleu_score`**: 计算机器翻译输出的BLEU分数。
    - **`nltk.translate.gale_church`**: 使用Gale-Church算法对齐双语语料。

这些板块涵盖了从基本的文本处理到复杂的语言理解和生成任务的广泛功能，帮助开发者和研究人员进行有效的自然语言处理和分析。在本次课程当中，我们要使用的板块是语料库板块（cropus），我们使用的数据集是从语料库板块中被导出的。

In [4]:
from nltk.corpus import sentence_polarity

当一个数据集被从语料库中import出来之后，我们可以使用sents()方法来调取出具体句子（虽然大部分时候是分割好的tokens），并使用.categories()方法来查看这个数据集所对应的标签类别：

In [5]:
sentence_polarity.sents() #非常便利，是已经分好的句子、每个句子里都是已分割好的tokens，无需我们再按照空格进行分割

[['simplistic', ',', 'silly', 'and', 'tedious', '.'], ["it's", 'so', 'laddish', 'and', 'juvenile', ',', 'only', 'teenage', 'boys', 'could', 'possibly', 'find', 'it', 'funny', '.'], ...]

In [12]:
sentence_polarity.categories()

['neg', 'pos']

In [14]:
#赋值，为后续展示句子做准备
sentences = sentence_polarity.sents()
label_type = sentence_polarity.categories()

要查看每个样本所对应的标签，需要特殊的指令：

In [15]:
from nltk.corpus import sentence_polarity

# 加载句子和标签
documents = [(list(sentence), category) for category in sentence_polarity.categories() 
             for sentence in sentence_polarity.sents(categories=category)]

# 打印前几个句子及其标签
for doc, label in documents[:10]:
    print('句子:', ' '.join(doc))
    print('标签:', label)

句子: simplistic , silly and tedious .
标签: neg
句子: it's so laddish and juvenile , only teenage boys could possibly find it funny .
标签: neg
句子: exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable .
标签: neg
句子: [garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation .
标签: neg
句子: a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .
标签: neg
句子: the story is also as unoriginal as they come , already having been recycled more times than i'd care to count .
标签: neg
句子: about the only thing to give the movie points for is bravado -- to take an entirely stale concept and push it through the audience's meat grinder one more time .
标签: neg
句子: not so much farcical as sour .
标签: neg
句子: unfortunately the story and the actors are served with a hack script .
标签: neg
句子: all the more disquieting for 

In [16]:
sentences.__len__() #一共有1w多个句子，每个句子里包含一串单词，是一个量级较小的数据集

10662

In [17]:
sentences[3] #可以通过索引的方式取出单一的句子

['[garbus]',
 'discards',
 'the',
 'potential',
 'for',
 'pathological',
 'study',
 ',',
 'exhuming',
 'instead',
 ',',
 'the',
 'skewed',
 'melodrama',
 'of',
 'the',
 'circumstantial',
 'situation',
 '.']

针对这样的数据，我们需要构建训练数据、测试数据集，并且将文本标签转化为数字；然而在进行所有的这些操作之前，我们需要构建该数据对应的词典和词汇表。

## 词汇表构建、词频筛选与句子编码

构建词汇表在自然语言处理任务中具有至关重要的作用，因为它直接影响到模型的训练效率和性能。一个高质量的词汇表能够有效地捕捉文本中的关键信息，减少噪音，同时提升模型的理解能力和处理速度。通过筛选出频率高、价值高的词汇，并过滤掉低频、无用的词汇，可以确保模型专注于学习有用的模式和特征，避免资源浪费和性能下降。词汇表的构建不仅决定了输入数据的表示方式，还对模型的泛化能力和应用效果产生深远影响。因此，构建一个精简而高效的词汇表是成功进行自然语言处理的基础和关键。

在构建词汇表后，我们将根据词汇表进行词典的构建、然后根据词典进行句子编码。这个任务流程通常是线性的，但是在本次的案例中，我们将一改往日“线性编程”的风格，直接为大家定义一个能够一步到位、一次性满足**1）构建词汇表、2）构建词典、3）根据词典对任意句子完成编码**这三个任务的综合类Vocab。由于打破了传统的线性构造，这个类也具有十分巧妙的编程设计，接下来我们就详细展开谈谈这个类的构建。

首先，在NLP任务中，虽然词汇表构建的流程是必须的，但是词汇表构建的来源却可以是多样的——例如，我们可能只有链接在一起的长文本text，也可能我们有已经分好词的Token列表——

In [19]:
text = "This is a sample text used for demonstrating tokenization. \
        Tokenization is the process of breaking down text into smaller\
        units such as words or phrases."

In [20]:
tokens = [
    "This", "is", "a", "sample", "text", "used", "for", "demonstrating", "tokenization",
    ".", "Tokenization", "is", "the", "process", "of", "breaking", "down", "text"
    , "into", "smaller", "units", "such", "as", "words", "or", "phrases", "."
]

In [21]:
text = "这是一个用于展示分词的示例文本。分词是将文本拆分成更小单位如词或短语的过程。"

In [22]:
tokens = [
    "这是", "一个", "用于", "展示", "分词", "的", "示例", "文本", "。", 
    "分词", "是", "将", "文本", "拆分", "成", "更", "小", "单位", "如", "词", 
    "或", "短语", "的", "过程", "。"
]

这两类输入数据都可以用来构建词汇表。**因此Vocab类的输入应该可以同时接纳这两种数据**。假设数据是token的话，那我们只需要在token的基础上关注一下“未知词”就可以了。

> **未知词是什么？**<br><br>
> 在进行深度学习算法预测的时候，只有在词汇表中的词才能够被进行编码，但是在预测中、可能会出现很多不常用的词、拼写错误的词、甚至可能因为训练数据集比较小、所以在测试数据中出现陌生的词。当出现这种情况时，编码就可能会发生错误，而且如果每次都给不常用词、或者拼写错误的词分配唯一的索引、可能会导致词汇表非常大且稀疏。因此在我们构建词汇表的时候，我们要使用字符串"\<unk>"（unknown）来代表模型在训练过程中从未遇到过的词，以减少词汇表的大小，提高模型的泛化能力。有了"\<unk>"的存在，模型就能够处理在训练中未遇到过的词。

但如果输入的数据是text而不是Token的话，就要先进行词频筛选、筛选出词频足够高的Tokens后，再处理未知词。

> **什么是词频筛选？**<br><br>词汇频率筛选在自然语言处理任务中具有重要意义，因为它能够有效地减少噪音，提高模型的训练效率和性能。通过统计每个词汇在文本中的出现频率，我们可以筛选出那些高频且有价值的词汇，避免将稀有词汇引入词汇表。低频词汇往往是拼写错误、特殊符号或不常见的词，这些词汇不仅增加了模型的复杂性，还可能对模型的学习过程产生干扰。通过过滤掉这些低频词汇，我们不仅能够简化词汇表，减少计算资源的消耗，还能使模型更专注于学习有用的词汇和模式，从而提升整体的表现。此外，较小的词汇表意味着更少的参数需要处理，训练速度也因此得到显著提升。因此，词汇频率筛选在构建高质量词汇表时至关重要，是提高模型效率和性能的关键步骤。

因此，Vocab的定义过程如下。首先，**Vocab类中有两个核心板块**，一个是所有类都必须定义的init，另一个是build方法，其中init中的流程用于接受原本就是Token格式的数据，直接处理未知词而build方法用于接受text数据，对text进行词频筛选、并将text处理成Tokens。

注意，在这个过程中，build方法并不负责处理未知词，而init方法并不负责进行词频筛选，但根据NLP处理的流程，任何Token都必须经过未知词处理，否则在构建词典的时候就会报错，因此当build方法将text处理成Tokens后，这个Tokens应该需要再放入init中进行未知词的处理。此时你会发现，如果将Vocab构建成一个具有递归性质的类，所有的问题和流程将迎刃而解，来看下面的代码：

### Vocab：递归的编码函数

In [ ]:
from collections import defaultdict, Counter

class Vocab:
    def __init__(self, tokens=None):
        # init的输入参数是Token
        # 构建两个变量，一个idx_to_token，一个是token_to_idx
        # idx_to_token是词汇表，包含了数据集中所有的单词
        # token_to_idx是词典，是词汇表 + 索引构成的结果
        self.idx_to_token = list()
        self.token_to_idx = dict()

        #如果输入了tokens（Tokens不为None）
        #就直接进行未知词操作
        #注意！这里的Token要求是一个【单一的token list】
        #也就是这个列表里只能有token本身，不能再包含其他内容
        #比如，一个list中包含了多个句子，每个句子都是按照token的方式排列的
        #那这个list就不属于【单一的token list】
        if tokens is not None:
            # 如果tokens中不包含"<unk>"这个词（未知词），则添加"<unk>"
            if "<unk>" not in tokens:
                tokens = tokens + ["<unk>"]
            # 遍历tokens，将每个token添加到idx_to_token列表，并在token_to_idx字典中映射其索引
            # 基于添加了未知词的Tokens，直接创造出词汇表和词典
            for token in tokens:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
            # 设置未知词的索引，将未知的词设置为一个单独的属性self.unk
            self.unk = self.token_to_idx['<unk>']

    #调用魔法命令classmethod，这个命令允许我们在不进行实例化的情况下使用类中的方法
    #build的输入参数与Vocab本身的init完全不同，因此我们可以运行它被单独调用
    @classmethod

    def build(cls, text, min_freq=1, reserved_tokens=None):
        # build，此时输入的参数有4个
        # cls是Vocab这个类本身，这魔法命令classmethod的要求
        # 有了cls就可以在不进行实例化的情况下直接调用build功能
        # text是需要构建词汇表和词典的文本，在这个文本上我们可以直接开始进行词频筛选
        # 注意！这个文本的范围很广泛，只要不是单一token list，都可以被认为是文本
        # min_freq是我们用于筛选的最小频率，低于该频率阈值的词会被删除
        # reserved_token是我们可以选择性输入的"通用词汇表"，假设text本身太短词太少的话
        # reserved_token可以帮助我们构建更大的词向量空间
        # 以上4个参数中只有text是必填的
        
        # 创建一个defaultdict字典，用于统计每个单词的出现频率
        token_freqs = defaultdict(int)
        # 遍历文本中的每个句子，统计每个单词的出现次数
        # 其中，单词使用变量token来代表
        for sentence in text:
            for token in sentence:
                # 不断保存到字典中的是——
                # 以token（词本身）作为键、词出现的频率作为值的键值对
                token_freqs[token] += 1
        
        # 创建一个空列表uniq_tokens，用于存储"<unk>"和输入用来保底的reserved_tokens
        uniq_tokens = ["<unk>"] + (reserved_tokens if reserved_tokens else [])
        
        # 将token_freqs中保存的词和词频进行循环
        # 除了"<unk>"之外，过滤掉出现次数少于min_freq的词
        # 并将没有被过滤掉的词打包到一个列表中
        # 这个列表uniq_tokens就是过滤后的Tokens列表
        uniq_tokens += [token for token, freq in token_freqs.items() if freq >= min_freq and token != "<unk>"]
        # 将过滤后的Tokens列表放入cls，也就是Vocab类中
        # 这个Token进入到Vocab类之后，会触发init，开始进入init中的流程
        # 因此，只要调用build方法，就可以从text构建一组token、并将这组token放入Vocab类
        # 这是这个类的“递归”所在，我们可以调用类中的方法来创造类所需的数据类型
        # 并在该方法的最后重启这个类
        return cls(uniq_tokens)

根据上面的代码，相信你已经发现Vocab这个类的神奇之处了。当我们的输入数据为Token时，我们可以直接调用Vocab类来同时生成词汇表和词典，当我们输入的数据是text时，我们可以调用build方法，build不仅能够直接将text转变为tokens，还能把tokens放入Vocab类、来生成词汇表和词典。Vocab因此能够同时包容text和tokens两种不同结构的数据，因此在使用这个类时，我们的具体调用方法是 ↓

#示例代码，不可运行

#当输入数据为Token时，调用流程（该流程的本质是实例化）：

vocab = Vocab(tokens)

#当输入数据为text时，调用流程（该流程的本质是通过@classmethod方法跳过实例化过程，直接调用类中的方法，不过build方法最终return的还是Vocab类，因此其本质相当于先处理text为token，再实例化）：

vocab = Vocab.build(text)

这两种方法下生成的vocab都是类，不过这个类中此时已经包括了生成好的 词汇表idx_to_token和词典token_to_idx。

需要注意的是，我们有提到init只能接受【单一token list】，这是指——

In [31]:
sentence_polarity.sents()
#我们此时使用的数据，虽然看起来是token但实际上是包含了多个tokens的列表
#因此无法被Vocab的init直接处理

[['simplistic', ',', 'silly', 'and', 'tedious', '.'], ["it's", 'so', 'laddish', 'and', 'juvenile', ',', 'only', 'teenage', 'boys', 'could', 'possibly', 'find', 'it', 'funny', '.'], ...]

但是init可以处理——

In [33]:
sentence_polarity.sents()[0] #这就是单一token list

['simplistic', ',', 'silly', 'and', 'tedious', '.']

### Vocab类的使用与循环

接下来，我们只需要写一系列的方法来调用我们的词汇表和词典、并根据词典将特定文字转化成编码后的结果就可以了。完整的Vocab类代码如下：

In [23]:
from collections import defaultdict, Counter

class Vocab:
    def __init__(self, tokens=None):
        # 初始化两个变量
        # idx_to_token是词汇表，包含了数据集中所有的单词
        # token_to_idx是词典，是词汇表 + 索引构成的结果
        self.idx_to_token = list()
        self.token_to_idx = dict()
        
        if tokens is not None:
            # 如果tokens中不包含"<unk>"（未知词），则添加"<unk>"
            if "<unk>" not in tokens:
                tokens = tokens + ["<unk>"]
            # 遍历tokens，将每个token添加到idx_to_token列表，并在token_to_idx字典中映射其索引
            # 同时创造出词汇表和词典
            for token in tokens:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
            # 设置未知词的索引，将未知的词设置为一个单独的属性self.unk
            self.unk = self.token_to_idx['<unk>']

    @classmethod
    def build(cls, text, min_freq=1, reserved_tokens=None):
        # 创建一个defaultdict，用于统计每个token的出现频率
        token_freqs = defaultdict(int)
        # 遍历文本中的每个句子，统计每个token的出现次数
        for sentence in text:
            for token in sentence:
                token_freqs[token] += 1
        # 创建一个列表uniq_tokens，包含"<unk>"和预留的tokens
        uniq_tokens = ["<unk>"] + (reserved_tokens if reserved_tokens else [])
        # 过滤掉出现次数少于min_freq的tokens，除了"<unk>"
        uniq_tokens += [token for token, freq in token_freqs.items() if freq >= min_freq and token != "<unk>"]
        # 返回一个新的Vocab实例
        return cls(uniq_tokens)

    #在已经生成词汇表和词典的基础上
    #我们可以对词汇表和词典进行各项操作 ↓
    def __len__(self):
        # 返回词汇表的大小
        return len(self.idx_to_token)

    def __getitem__(self, token):
        # 获取token对应的索引，如果不存在则返回未知词的索引
        return self.token_to_idx.get(token, self.unk)

    def convert_tokens_to_ids(self, tokens):
        # 将token列表转换为索引列表（也就是将文字进行编码）
        return [self[token] for token in tokens]

    def convert_ids_to_tokens(self, indices):
        # 将索引列表转换为token列表（也就是根据编码、找到相应的token）
        return [self.idx_to_token[index] for index in indices]

在这段代码中，你还可以增加许多其他的操作，例如直接让vocab类为我们返回idx_to_token和token_to_idx这两个结果，便于你查看词汇表和词典，也可以针对词汇表和词典进行更加丰富的操作。在这里，最关键的功能就是将token转换成索引、即将文字进行编码的过程。接下来我们可以来测试一下该段代码：

In [30]:
sentence_polarity.sents()
#我们此时使用的数据，虽然看起来是token但实际上是包含了多个tokens的列表
#因此无法被Vocab的init直接处理

[['simplistic', ',', 'silly', 'and', 'tedious', '.'], ["it's", 'so', 'laddish', 'and', 'juvenile', ',', 'only', 'teenage', 'boys', 'could', 'possibly', 'find', 'it', 'funny', '.'], ...]

In [27]:
vocab = Vocab(sentence_polarity.sents()) #会发现，因为列表中还包含列表，所以无法处理

TypeError: unhashable type: 'list'

In [34]:
vocab = Vocab.build(sentence_polarity.sents()) #但是build却可以把这个列表当做text来处理

In [36]:
vocab #此时生成的vocab是一个类

In [37]:
sentence = sentence_polarity.sents(categories='pos')[0]

In [39]:
#将一个句子根据词典转变成索引
vocab.convert_tokens_to_ids(sentence)

[23,
 2444,
 61,
 9851,
 76,
 308,
 23,
 1664,
 14509,
 496,
 219,
 14510,
 219,
 4,
 27,
 175,
 363,
 76,
 29,
 32,
 5884,
 201,
 7984,
 73,
 5354,
 4219,
 2,
 14511,
 1204,
 2701,
 25,
 2184,
 14512,
 6]

In [40]:
#我们还可以定义其他的方法，例如——

def save_vocab(vocab, path):
    # 将词汇表保存到指定路径的文件中
    with open(path, 'w') as writer:
        writer.write("\n".join(vocab.idx_to_token))

def read_vocab(path):
    # 从指定路径的文件中读取词汇表
    with open(path, 'r') as f:
        tokens = f.read().split('\n')
    # 返回一个新的Vocab实例，初始化它的词汇表
    return Vocab(tokens)

在接下来的案例实际过程中，我们虽然不会使用到这两个函数，但这两个函数可以帮助你进一步拓展你的函数使用方法。整个vocab类的编排十分巧妙，未来你依然可以借助这个思路来编排你的词典构建函数。

## 文本数据处理与标签创建

### 循环创建训练/测试集

现在已经有了能够根据文字本身构建词汇表、词典、并将数据根据词典进行编码的类Vocab，接下里我们将利用这个类来创建相应的训练、测试数据集。

In [42]:
from nltk.corpus import sentence_polarity

In [43]:
#赋值，为后续展示句子做准备
sentences = sentence_polarity.sents()
label_type = sentence_polarity.categories()

In [44]:
sentences.__len__() #一共有1w多个句子，每个句子里包含一串单词，是一个量级较小的数据集

10662

由于polarity数据中的数据量十分庞大，我们将其中的前4000个句子作为训练集、4000-10662个句子作为测试集来构建数据集。在NLP的任务中，训练数据的数量比测试数据小很多是正常现象。在分割数据集的过程中，我们是直接使用列表推导式不断从positive和negative两个子集中取出相应的句子，并对每一个句子按照字典的方式进行编码。我们定义这个编码函数为load_sentences_polarity，具体代码如下：

In [45]:
def load_sentence_polarity():
    # 导入NLTK库中的sentence_polarity模块
    from nltk.corpus import sentence_polarity

    # 创建一个词汇表vocab
    # 词汇表是在字典之前、先对句子中出现的所有不重复的词进行统计的表单
    # 先构建词汇表vocab，再对vocab进行编码就可以构成字典
    vocab = Vocab.build(sentence_polarity.sents())

    # 构建训练数据集，将正面情感的句子标记为0，取前4000个正面句子
    # 负面情感的句子标记为1，取前4000个负面句子
    train_data = [(vocab.convert_tokens_to_ids(sentence), 0)
                  for sentence in sentence_polarity.sents(categories='pos')[:4000]] \
    + [(vocab.convert_tokens_to_ids(sentence), 1)
            for sentence in sentence_polarity.sents(categories='neg')[:4000]]

    # 构建测试数据集，使用剩余的正面情感句子，标记为0
    # 使用剩余的负面情感句子，标记为1
    test_data = [(vocab.convert_tokens_to_ids(sentence), 0)
                 for sentence in sentence_polarity.sents(categories='pos')[4000:]] \
        + [(vocab.convert_tokens_to_ids(sentence), 1)
            for sentence in sentence_polarity.sents(categories='neg')[4000:]]

    # 返回训练数据、测试数据和vocab类本身
    return train_data, test_data, vocab

### 布尔掩码与有效时间步管理

在处理自然语言处理（NLP）任务时，输入的数据常常是可变长度的序列。例如，一些句子可能只有几个单词，而另一些句子可能有几十个单词。在这种情况下，我们需要一种方法来标记哪些时间步是有效的，即哪些时间步对应的是输入序列中的实际单词，哪些时间步是填充（padding）字符。

有效的时间步是指那些在序列中实际包含有意义信息的时间步。例如，对于一个句子 "I love NLP"，其有效的时间步是 "I", "love", "NLP"，而如果这个句子被填充到一个固定长度，比如 6 个单词的长度，填充后的句子可能是 "I love NLP <pad> <pad> <pad>"。在这种情况下，只有前 3 个时间步是有效的，后面的 3 个时间步是填充字符 "<pad>"。

因此在这里，我们需要生成一个布尔掩码，用于标记每个序列中有效的时间步。这个掩码在处理可变长度序列时非常有用，例如在计算模型损失或进行序列模型的计算时，可以用掩码忽略填充字符的位置。

In [47]:
def length_to_mask(lengths):
    # 获取输入张量lengths中的最大长度
    max_len = torch.max(lengths)
    
    # 创建一个大小为 (batch_size, max_len) 的布尔掩码张量
    # torch.arange(max_len) 生成一个从 0 到 max_len-1 的张量
    # expand(lengths.shape[0], max_len) 将这个张量扩展到 (batch_size, max_len)
    # lengths.unsqueeze(1) 将 lengths 张量的维度扩展，以便进行比较
    # 结果是一个布尔掩码，标记哪些位置是有效的时间步
    mask = torch.arange(max_len, device=lengths.device).expand(lengths.shape[0], max_len) < lengths.unsqueeze(1)
    
    # 返回生成的布尔掩码张量
    return mask

假设我们有以下序列长度：

In [49]:
lengths = torch.tensor([3, 2, 4])

In [51]:
mask = length_to_mask(lengths)
print(mask)

tensor([[ True,  True,  True, False],
        [ True,  True, False, False],
        [ True,  True,  True,  True]])


在这个例子中，生成的掩码表明：

第一个序列的前 3 个时间步是有效的（长度为 3），第四个时间步是填充。<br><br>
第二个序列的前 2 个时间步是有效的（长度为 2），后两个时间步是填充。<br><br>
第三个序列的前 4 个时间步都是有效的（长度为 4）。

length_to_mask函数通过生成布尔掩码来标记有效的时间步，使得在处理可变长度序列时可以轻松地忽略填充字符的位置。这在训练和评估序列模型时尤为重要，因为填充字符不包含有意义的信息，不应影响模型的计算和学习过程。

## 训练与预测流程

### 架构构建与掩码设置

在有了数据集之后，现在让我们来代码定义并实现一个基于 Transformer 的文本分类模型，这个模型既可以适用于情感分类，也可以适用于词性标注任务。首先，我们使用Dataset类封装了适用于pytorch的数据集，使数据可以方便地被pytorch以及dataloader等类进行批处理；然后定义了一个函数来处理批次数据，对序列进行填充；接着实现了位置编码以帮助模型捕捉序列信息；最后构建了一个包含词嵌入层、位置编码层、Transformer 编码器层和输出层的 Transformer 模型，用于对文本进行分类。

In [59]:
!pip install --upgrade tqdm

     -------------------------------------- 57.6/57.6 kB 606.6 kB/s eta 0:00:00
   ---------------------------------------- 78.3/78.3 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3


保持tqdm为最新版本，确保与jupyterlab或你的其他代码运行设备兼容，这样可以减少tqdm的报错和报警告概率。

In [57]:
# tqdm是一个Python模块，能以进度条的方式显式迭代的进度
from tqdm.auto import tqdm

# 定义TransformerDataset类，用于处理数据集
class TransformerDataset(Dataset):
    def __init__(self, data):
        # 初始化数据集，将传入的数据保存在实例变量data中
        self.data = data

    def __len__(self):
        # 返回数据集的大小
        return len(self.data)

    def __getitem__(self, i):
        # 根据索引i获取数据集中的第i个样本
        return self.data[i]

# 定义collate_fn函数，用于在DataLoader中对一个batch的数据进行处理
def collate_fn(examples):
    # 获取每个样本的长度，并将其转换为张量
    lengths = torch.tensor([len(ex[0]) for ex in examples])
    # 将每个样本的输入部分转换为张量
    inputs = [torch.tensor(ex[0]) for ex in examples]
    # 将每个样本的目标部分转换为长整型张量
    targets = torch.tensor([ex[1] for ex in examples], dtype=torch.long)
    # 对batch内的样本进行padding，使其具有相同长度
    inputs = pad_sequence(inputs, batch_first=True)
    # 返回处理后的输入、长度和目标
    return inputs, lengths, targets

# 定义PositionalEncoding类，用于为输入添加位置信息
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=512):
        # 调用父类的构造函数
        super(PositionalEncoding, self).__init__()
        
        # 创建一个大小为(max_len, d_model)的全零张量
        pe = torch.zeros(max_len, d_model)
        # 创建一个从0到max_len-1的序列，并扩展为(max_len, 1)的张量
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 计算位置编码公式中的除数部分
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # 将位置编码的奇数部分应用sin函数
        pe[:, 0::2] = torch.sin(position * div_term)
        # 将位置编码的偶数部分应用cos函数
        pe[:, 1::2] = torch.cos(position * div_term)
        # 增加一个维度并进行转置
        pe = pe.unsqueeze(0).transpose(0, 1)
        # 将位置编码张量注册为模型的缓冲区
        self.register_buffer('pe', pe)

    def forward(self, x):
        # 将位置编码添加到输入x中
        x = x + self.pe[:x.size(0), :]
        return x

# 定义Transformer类，用于实现Transformer模型
class Transformer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_class,
                 dim_feedforward=512, num_head=2, num_layers=2, dropout=0.1, max_len=128, activation: str = "relu"):
        # 调用父类的构造函数
        super(Transformer, self).__init__()
        # 词嵌入层
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 位置编码层
        self.position_embedding = PositionalEncoding(embedding_dim, dropout, max_len)
        # 编码层：使用Transformer
        encoder_layer = nn.TransformerEncoderLayer(hidden_dim, num_head
                                                   , dim_feedforward
                                                   , dropout, activation
                                                  , batch_first = False)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        # 输出层
        self.output = nn.Linear(hidden_dim, num_class)

    def forward(self, inputs, lengths):
        # 转置输入张量
        inputs = torch.transpose(inputs, 0, 1)
        # 获取词嵌入表示
        hidden_states = self.embeddings(inputs)
        # 添加位置编码
        hidden_states = self.position_embedding(hidden_states)
        # 生成注意力掩码
        attention_mask = length_to_mask(lengths) == False
        # 通过Transformer编码器进行处理
        hidden_states = self.transformer(hidden_states, src_key_padding_mask=attention_mask)
        # 取出第一个时间步的隐藏状态
        hidden_states = hidden_states[0, :, :]
        # 通过输出层得到分类结果
        output = self.output(hidden_states)
        # 计算对数概率
        log_probs = F.log_softmax(output, dim=1)
        return log_probs

接下来定义数据加载、模型构建、训练和测试等步骤的具体代码。

首先，设置了模型的参数，包括词嵌入维度、隐藏层维度、类别数量、批次大小和训练轮数。接着，通过 load_sentence_polarity 函数加载训练数据、测试数据和词汇表，并创建对应的数据集和数据加载器。然后，选择设备（GPU或CPU）并实例化 Transformer 模型，将模型加载到选定的设备上。进入训练过程时，使用负对数似然损失函数和 Adam 优化器，在每个训练轮次中，遍历训练数据，进行前向传播、计算损失、反向传播和参数更新，并记录每轮次的总损失。在测试过程中，通过遍历测试数据，禁用梯度计算进行前向传播，计算预测准确率，最终输出在测试集上的总体准确率。整个流程通过 tqdm 模块显示训练和测试的进度。

In [61]:
embedding_dim = 128  # 词嵌入的维度为128
hidden_dim = 128  # 隐藏层的维度为128
num_class = 2  # 类别数量为2（例如正面和负面）
batch_size = 32  # 每个批次的大小为32
num_epoch = 10  # 训练的轮数为10

# 加载数据
train_data, test_data, vocab = load_sentence_polarity()  # 调用函数加载训练数据和测试数据，以及词汇表
train_dataset = TransformerDataset(train_data)  # 创建训练数据集
test_dataset = TransformerDataset(test_data)  # 创建测试数据集
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)  # 创建训练数据加载器
test_data_loader = DataLoader(test_dataset, batch_size=1, collate_fn=collate_fn, shuffle=False)  # 创建测试数据加载器

# 加载模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 选择设备（GPU或CPU）
model = Transformer(len(vocab), embedding_dim, hidden_dim, num_class)  # 实例化Transformer模型
model.to(device)  # 将模型加载到GPU中（如果已经正确安装）

# 训练过程
nll_loss = nn.NLLLoss()  # 使用负对数似然损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用Adam优化器，学习率为0.001

model.train()  # 设置模型为训练模式
for epoch in range(num_epoch):  # 遍历每个训练轮次
    total_loss = 0  # 初始化总损失为0
    for batch in tqdm(train_data_loader, desc=f"Training Epoch {epoch}",disable=True):  # 遍历每个批次，显示训练进度
        inputs, lengths, targets = [x.to(device) for x in batch]  # 将输入、长度和目标数据移动到设备上
        log_probs = model(inputs, lengths)  # 前向传播计算对数概率
        loss = nll_loss(log_probs, targets)  # 计算损失
        optimizer.zero_grad()  # 清零梯度
        loss.backward()  # 反向传播计算梯度
        optimizer.step()  # 更新模型参数
        total_loss += loss.item()  # 累加损失
    print(f"Loss: {total_loss:.2f}")  # 输出每个轮次的总损失

# 测试过程
acc = 0  # 初始化准确率计数器
for batch in tqdm(test_data_loader, desc=f"Testing"):  # 遍历每个测试批次，显示测试进度
    inputs, lengths, targets = [x.to(device) for x in batch]  # 将输入、长度和目标数据移动到设备上
    with torch.no_grad():  # 禁用梯度计算
        output = model(inputs, lengths)  # 前向传播计算输出
        acc += (output.argmax(dim=1) == targets).sum().item()  # 计算预测正确的数量并累加

# 输出在测试集上的准确率
print(f"Acc: {acc / len(test_data_loader):.2f}")  # 计算并输出测试集上的准确率

Loss: 175.01
Loss: 160.55
Loss: 136.09
Loss: 108.17
Loss: 85.01
Loss: 64.21
Loss: 47.93
Loss: 38.23
Loss: 30.87
Loss: 32.25


Testing:   0%|          | 0/2662 [00:00<?, ?it/s]

Acc: 0.68


损失在稳步下降中，我们还可以继续训练、继续调整，对模型进行进一步的提升。

# Transformer案例实战 之 词性标注

词性标注和情感分类是自然语言处理领域中的两个重要任务，虽然它们的目的和方法有所不同，但也存在一些相似之处。最大的相似点就是，词性标注与情感分类可以共享一整套完整的预处理、训练代码，在情感分类代码的基础上，我们仅仅需要将情感分类的数据集修改为词性标注的数据集，就可以顺利执行词性标注的任务。

**因此，在阅读词性标注任务之前，请一定一定要完整阅读情感分类任务的实战部分！**<br><br>
**因此，在阅读词性标注任务之前，请一定一定要完整阅读情感分类任务的实战部分！**<br><br>
**因此，在阅读词性标注任务之前，请一定一定要完整阅读情感分类任务的实战部分！**

词性标注的任务是为每个单词分配一个词性标签（如名词、动词、形容词等），从而揭示句子的语法结构。这一任务通常需要利用上下文信息来准确判定词性，例如确定“book”在句子中是作为名词还是动词。另一方面，情感分类的任务是分析整个文本的情感倾向，并将其归类为正面、负面或中性等情感类别。这需要模型能够理解文本的整体语义和情感表达。

相似之处在于，两者都依赖于对文本的深度理解，都需要处理自然语言中的复杂结构和语境信息，并且都可以使用类似的模型架构，例如Transformer。Transformer模型通过自注意力机制能够有效捕捉文本中的长程依赖和上下文关系，这对于准确进行词性标注和情感分类都至关重要。此外，这两个任务都可以使用标注好的数据集进行监督学习，以提高模型的准确性。

然而，它们也有显著的不同。词性标注更关注句子的语法和结构，对每个单词进行细粒度的分析；而情感分类则关注文本的语义和情感，对整个文本进行整体分析。因此，词性标注通常是一个序列标注任务，而情感分类是一个文本分类任务。词性标注需要在词汇级别进行预测，而情感分类则在句子或文档级别进行预测。尽管如此，这两个任务都在自然语言处理的不同应用中发挥着重要作用，且相互补充，共同提升了机器对自然语言的理解和处理能力。

## 词性标注的意义与用途

词性分类（词性标注）的任务在自然语言处理（NLP）领域具有重要的意义和广泛的应用。以下是词性分类任务的几个关键意义：

1. **理解语法结构**<br><br>
> 词性标注是理解和分析句子语法结构的基础。通过为每个单词分配适当的词性标签（如名词、动词、形容词等），可以揭示句子的基本语法结构，帮助理解句子成分之间的关系。例如，在句子中区分主语、谓语和宾语的角色。

2. **提高自然语言处理任务的性能**
> 词性标注对许多NLP任务的性能有直接影响。例如：<br><br>
> 命名实体识别（NER）：识别专有名词如人名、地名和组织名称，词性信息可以显著提高NER的准确性。<br><br>
> 依存句法解析：理解句子中单词之间的依赖关系，词性信息是解析句法结构的重要特征。<br><br>
> 情感分析：确定文本的情感倾向时，识别形容词和副词等词性有助于更准确地分析情感表达。<br><br>
> 机器翻译：在翻译过程中，词性标注帮助理解源语言的语法结构，从而生成更准确和流畅的目标语言翻译。

3. **提供词汇信息**<br><br>
> 词性标注提供了关于词汇的丰富信息，有助于构建词汇资源和词典。例如，基于词性标注的数据可以用于生成带有词性标签的词典，帮助词典用户理解单词的用法和语法功能。

4. **预处理步骤**<br><br>
> 词性标注通常作为其他高级NLP任务的预处理步骤。例如，在问答系统中，理解用户输入的语法结构有助于更好地解析用户的意图并生成准确的答案。

5. **语言学习和教学**<br><br>
> 词性标注在语言学习和教学中也具有重要意义。自动词性标注系统可以帮助语言学习者理解句子的语法结构，提供即时的反馈和辅助教学。

6. **应用实例**<br><br>
> 搜索引擎：改进查询解析，理解用户搜索意图。<br><br>
> 文本摘要：识别关键信息和句子成分，生成简洁准确的摘要。<br><br>
> 语音识别：识别口语中的词性，改进转录文本的准确性。

## 词性标注所使用的数据集与标签

在词性标准任务中，我们所使用的数据集是treebank：

In [ ]:
def load_treebank():
    # 从nltk.corpus中导入treebank模块
    from nltk.corpus import treebank
    # 获取treebank中的标注句子，并分别提取句子和词性标签
    sents, postags = zip(*(zip(*sent) for sent in treebank.tagged_sents()))

    # 使用句子构建词汇表，预留<pad>标记
    vocab = Vocab.build(sents, reserved_tokens=["<pad>"])

    # 使用词性标签构建标签词汇表
    tag_vocab = Vocab.build(postags)

    # 构建训练数据集，将句子和对应的标签编码为数字
    train_data = [(vocab.convert_tokens_to_ids(sentence), tag_vocab.convert_tokens_to_ids(tags)) for sentence, tags in zip(sents[:3000], postags[:3000])]
    # 构建测试数据集，将句子和对应的标签编码为数字
    test_data = [(vocab.convert_tokens_to_ids(sentence), tag_vocab.convert_tokens_to_ids(tags)) for sentence, tags in zip(sents[3000:], postags[3000:])]

    # 返回训练数据集、测试数据集、词汇表和标签词汇表
    return train_data, test_data, vocab, tag_vocab

Treebank 语料库是一个包含结构化和标注好的文本数据集，通常用于训练和测试各种自然语言处理模型。具体来说，Treebank 语料库主要提供以下几种类型的数据：

1. 词性标注（POS Tagging）：Treebank 中的每个单词都带有词性标签，表示这个单词在句子中的语法角色（如名词、动词、形容词等）。

2. 句法结构（Syntactic Structure）：Treebank 包含句法树，表示句子的语法结构。每个句子都被解析成树状结构，展示了各个词语和短语之间的语法关系。

3. 依存关系（Dependency Relations）：在一些 Treebank 版本中，提供了词语之间的依存关系，表示哪个词是哪个词的修饰语或主语等。

In [2]:
import nltk

#我们可以将nltk中所带的数据下载到本地来进行调用
#这个过程可能需要梯子，且仅限美国IP
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Shuyu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\treebank.zip.


True

In [3]:
from nltk.corpus import treebank

# 加载 Treebank 中的标注句子
tagged_sentences = treebank.tagged_sents()

# 打印前两个标注句子
for sentence in tagged_sentences[:2]:
    print(sentence)

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
[('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')]


你现在看到的元组就是数据集中不同的词和词性标注：

NNP：专有名词（单数）（Proper noun, singular）<br><br>
CD：基数词（Cardinal number）<br><br>
NNS：名词（复数）（Noun, plural）<br><br>
JJ：形容词（Adjective）<br><br>
MD：情态动词（Modal）<br><br>
VB：动词原形（Verb, base form）<br><br>
DT：限定词（Determiner）<br><br>
NN：名词（单数或不可数）（Noun, singular or mass）<br><br>
IN：介词或从属连词（Preposition or subordinating conjunction）<br><br>
.：句号（Period, full stop）<br><br>
VBZ：动词第三人称单数现在时（Verb, 3rd person singular present）<br><br>
VBG：动词现在分词或动名词（Verb, gerund or present participle）

例如，这个句子：

In [4]:
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

Pierre 和 Vinken 是专有名词（NNP）<br><br>
61 是基数词（CD）<br><br>
years 是复数名词（NNS）<br><br>
old 是形容词（JJ）<br><br>
will 是情态动词（MD）<br><br>
join 是动词原形（VB）<br><br>
the 是限定词（DT）<br><br>
board 是名词单数（NN）<br><br>
as 是介词或从属连词（IN）<br><br>
nonexecutive 是形容词（JJ）<br><br>
director 是名词单数（NN）<br><br>
Nov. 是专有名词（NNP）<br><br>
29 是基数词（CD）<br><br>
. 是句号（.）

在词性标注领域，有许多著名的数据集用于训练和评估自然语言处理模型。以下是一些在英文和中文词性标注中常用的著名数据集：

**英文词性标注数据集**——
> Penn Treebank：这是最著名的英语词性标注数据集之一，包含了丰富的词性标注信息，被广泛用于各种NLP任务。<br><br>
> Brown Corpus：一个多领域的语料库，包括新闻、小说、学术文章等，包含了详细的词性标注信息。<br><br>
> WSJ (Wall Street Journal) Corpus：包含来自《华尔街日报》的文章，常用于词性标注和句法解析任务。

**中文词性标注数据集**——
> 人民日報语料库 (People's Daily Corpus)：包含大量的中文新闻文本，广泛用于中文词性标注和其他NLP任务。<br><br>
> Penn Chinese Treebank：这是中文版本的 Penn Treebank，包含词性标注和句法结构信息，被广泛用于中文NLP研究。<br><br>
> PKU's People's Daily Corpus：由北京大学提供的中文语料库，包含大量的标注数据，适用于中文词性标注任务。<br><br>
> Chinese GigaWord Corpus：一个大型的中文文本语料库，包含新闻、文章等，广泛用于词性标注和其他NLP任务。<br><br>
> Sinica Treebank：由台湾中央研究院提供的中文语料库，包含详细的词性标注和句法结构信息。

**其他资源**
> 除了上述数据集，以下资源也经常用于词性标注和其他NLP任务：<br><br>
> Universal Dependencies (UD)：一个跨语言的树库，包含了多个语言的词性标注和句法依赖信息。中文的UD数据集也非常丰富，适用于多种NLP任务。<br><br>
> SIGHAN Bakeoff：由SIGHAN（国际汉语处理会议）提供的多次中文处理评测比赛，包含词性标注、分词等多种任务的数据集。

Penn Chinese Treebank 示例：

In [9]:
[('我', 'PN'), ('喜欢', 'VV'), ('自然语言处理', 'NN'), ('。', 'PU')]

[('我', 'PN'), ('喜欢', 'VV'), ('自然语言处理', 'NN'), ('。', 'PU')]

人民日報语料库 示例：

In [10]:
[('中国', 'NR'), ('是', 'VC'), ('一个', 'DT'), ('伟大', 'JJ'), ('的', 'DEG'), ('国家', 'NN'), ('。', 'PU')]

[('中国', 'NR'),
 ('是', 'VC'),
 ('一个', 'DT'),
 ('伟大', 'JJ'),
 ('的', 'DEG'),
 ('国家', 'NN'),
 ('。', 'PU')]

## 词性标注代码流程与训练流程

**在使用词性标注代码之前，请一定一定要完整学习情感分类任务的实战部分！**<br><br>
**在使用词性标注代码之前，请一定一定要完整学习情感分类任务的实战部分！**<br><br>
**在使用词性标注代码之前，请一定一定要完整学习情感分类任务的实战部分！**

In [18]:
import math
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from collections import defaultdict

In [19]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Shuyu\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [20]:
from collections import defaultdict, Counter

class Vocab:
    def __init__(self, tokens=None):
        self.idx_to_token = list()
        self.token_to_idx = dict()

        if tokens is not None:
            if "<unk>" not in tokens:
                tokens = tokens + ["<unk>"]
            for token in tokens:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
            self.unk = self.token_to_idx['<unk>']

    @classmethod
    def build(cls, text, min_freq=1, reserved_tokens=None):
        token_freqs = defaultdict(int)
        for sentence in text:
            for token in sentence:
                token_freqs[token] += 1
        uniq_tokens = ["<unk>"] + (reserved_tokens if reserved_tokens else [])
        uniq_tokens += [token for token, freq in token_freqs.items() \
                        if freq >= min_freq and token != "<unk>"]
        return cls(uniq_tokens)

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, token):
        return self.token_to_idx.get(token, self.unk)

    def convert_tokens_to_ids(self, tokens):
        return [self[token] for token in tokens]

    def convert_ids_to_tokens(self, indices):
        return [self.idx_to_token[index] for index in indices]


def save_vocab(vocab, path):
    with open(path, 'w') as writer:
        writer.write("\n".join(vocab.idx_to_token))


def read_vocab(path):
    with open(path, 'r') as f:
        tokens = f.read().split('\n')
    return Vocab(tokens)

In [21]:
def load_sentence_polarity():
    from nltk.corpus import sentence_polarity

    vocab = Vocab.build(sentence_polarity.sents())

    train_data = [(vocab.convert_tokens_to_ids(sentence), 0)
                  for sentence in sentence_polarity.sents(categories='pos')[:4000]] \
        + [(vocab.convert_tokens_to_ids(sentence), 1)
            for sentence in sentence_polarity.sents(categories='neg')[:4000]]

    test_data = [(vocab.convert_tokens_to_ids(sentence), 0)
                 for sentence in sentence_polarity.sents(categories='pos')[4000:]] \
        + [(vocab.convert_tokens_to_ids(sentence), 1)
            for sentence in sentence_polarity.sents(categories='neg')[4000:]]

    return train_data, test_data, vocab

def length_to_mask(lengths):
    max_len = torch.max(lengths)
    mask = torch.arange(max_len, device=lengths.device).expand(lengths.shape[0], max_len) < lengths.unsqueeze(1)
    return mask

def load_treebank():
    from nltk.corpus import treebank
    sents, postags = zip(*(zip(*sent) for sent in treebank.tagged_sents()))

    vocab = Vocab.build(sents, reserved_tokens=["<pad>"])

    tag_vocab = Vocab.build(postags)

    train_data = [(vocab.convert_tokens_to_ids(sentence), tag_vocab.convert_tokens_to_ids(tags)) for sentence, tags in zip(sents[:3000], postags[:3000])]
    test_data = [(vocab.convert_tokens_to_ids(sentence), tag_vocab.convert_tokens_to_ids(tags)) for sentence, tags in zip(sents[3000:], postags[3000:])]

    return train_data, test_data, vocab, tag_vocab

In [23]:
#tqdm是一个Pyth模块，能以进度条的方式显式迭代的进度
from tqdm.auto import tqdm

class TransformerDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, i):
        return self.data[i]

def collate_fn(examples):
    lengths = torch.tensor([len(ex[0]) for ex in examples])
    inputs = [torch.tensor(ex[0]) for ex in examples]
    targets = [torch.tensor(ex[1]) for ex in examples]
    # 对batch内的样本进行padding，使其具有相同长度
    inputs = pad_sequence(inputs, batch_first=True, padding_value=vocab["<pad>"])
    targets = pad_sequence(targets, batch_first=True, padding_value=vocab["<pad>"])
    return inputs, lengths, targets, inputs != vocab["<pad>"]

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=512):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term2 = torch.pow(torch.tensor(10000.0), torch.arange(0, d_model, 2).float() / d_model)
        div_term1 = torch.pow(torch.tensor(10000.0), torch.arange(1, d_model, 2).float() / d_model)
        # 高级切片方式，即从0开始，两个步长取一个。即奇数和偶数位置赋值不一样。直观来看就是每一句话的
        pe[:, 0::2] = torch.sin(position * div_term2)
        pe[:, 1::2] = torch.cos(position * div_term1)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_class,
                 dim_feedforward=512, num_head=2, num_layers=2, dropout=0.1, max_len=512, activation: str = "relu"):
        super(Transformer, self).__init__()
        # 词嵌入层
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.position_embedding = PositionalEncoding(embedding_dim, dropout, max_len)
        # 编码层：使用Transformer
        encoder_layer = nn.TransformerEncoderLayer(hidden_dim, num_head, dim_feedforward, dropout, activation)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        # 输出层
        self.output = nn.Linear(hidden_dim, num_class)

    def forward(self, inputs, lengths):
        inputs = torch.transpose(inputs, 0, 1)
        hidden_states = self.embeddings(inputs)
        hidden_states = self.position_embedding(hidden_states)
        attention_mask = length_to_mask(lengths) == False
        hidden_states = self.transformer(hidden_states, src_key_padding_mask=attention_mask).transpose(0, 1)
        logits = self.output(hidden_states)
        log_probs = F.log_softmax(logits, dim=-1)
        return log_probs

embedding_dim = 128
hidden_dim = 128
batch_size = 32
num_epoch = 10

#加载数据
train_data, test_data, vocab, pos_vocab = load_treebank()
train_dataset = TransformerDataset(train_data)
test_dataset = TransformerDataset(test_data)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=1, collate_fn=collate_fn, shuffle=False)

num_class = len(pos_vocab)

#加载模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Transformer(len(vocab), embedding_dim, hidden_dim, num_class)
model.to(device) #将模型加载到GPU中（如果已经正确安装）

#训练过程
nll_loss = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) #使用Adam优化器

model.train()
for epoch in range(num_epoch):
    total_loss = 0
    for batch in tqdm(train_data_loader, desc=f"Training Epoch {epoch}",disable=True):
        inputs, lengths, targets, mask = [x.to(device) for x in batch]
        log_probs = model(inputs, lengths)
        loss = nll_loss(log_probs[mask], targets[mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Loss: {total_loss:.2f}")

#测试过程
acc = 0
total = 0
for batch in tqdm(test_data_loader, desc=f"Testing"):
    inputs, lengths, targets, mask = [x.to(device) for x in batch]
    with torch.no_grad():
        output = model(inputs, lengths)
        acc += (output.argmax(dim=-1) == targets)[mask].sum().item()
        total += mask.sum().item()

#输出在测试集上的准确率
print(f"Acc: {acc / total:.2f}")

D:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Loss: 167.08
Loss: 95.58
Loss: 70.24
Loss: 52.22
Loss: 39.12
Loss: 29.26
Loss: 22.06
Loss: 17.06
Loss: 12.93
Loss: 10.51


Testing:   0%|          | 0/914 [00:00<?, ?it/s]

Acc: 0.85
